## Setup

In [5]:
training <- read.csv('train.csv')
test <- read.csv('test.csv')

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'gridExtra'

The following object is masked from 'package:dplyr':

    combine



In [ ]:
library(tidyverse)
library(gridExtra)

In [2]:
augmentRightTurn <- function(dataset) {
    isRightTurn <- function(entry, exit) {
        rightexit <- list(
            N = c("NW", "W", "SW"),
            NW = c("W", "SW", "S"),
            W = c("SW", "S", "SE"),
            SW = c("S", "SE", "E"),
            S = c("SE", "E", "NE"),
            SE = c("E", "NE", "N"),
            E = c("NE", "N", "NW"),
            NE = c("N", "NW", "W")
        )
        exit %in% rightexit[[entry]]
    }
    vIsRightTurn <- Vectorize(isRightTurn)
    dataset %>% 
        mutate(RightTurn = vIsRightTurn(EntryHeading, ExitHeading)) %>%
        group_by(IntersectionId, EntryHeading) %>%
        summarize(RightTurnAllowed = max(RightTurn)) %>%
        inner_join(dataset) %>%
        mutate(RightTurn = ifelse(vIsRightTurn(EntryHeading, ExitHeading), 1, 0))
}

In [3]:
augmentLeftTurn <- function(dataset) {
    isLeftTurn <- function(entry, exit) {
        leftexit <- list(
            N = c("NE", "E", "SE"),
            NW = c("N", "NE", "E"),
            W = c("NW", "N", "NE"),
            SW = c("W", "NW", "N"),
            S = c("SW", "W", "NW"),
            SE = c("S", "SW", "W"),
            E = c("SE", "S", "SW"),
            NE = c("E", "SE", "S")
        )
        exit %in% leftexit[[entry]]
    }
    vIsLeftTurn <- Vectorize(isLeftTurn)
    dataset %>% 
        mutate(LeftTurn = vIsLeftTurn(EntryHeading, ExitHeading)) %>%
        group_by(IntersectionId, EntryHeading) %>%
        summarize(LeftTurnAllowed = max(LeftTurn)) %>%
        inner_join(dataset) %>%
        mutate(LeftTurn = ifelse(vIsLeftTurn(EntryHeading, ExitHeading), 1, 0))
}

## Transformations and Feature Engineering

In [6]:
augTraining <- augmentRightTurn(training)
augTraining <- augmentLeftTurn(augTraining)

Joining, by = c("IntersectionId", "EntryHeading")
Joining, by = c("IntersectionId", "EntryHeading")


In [7]:
transformedTrain <- augTraining %>% select(IntersectionId,LeftTurnAllowed, RightTurnAllowed, RightTurn, LeftTurn, Latitude,Longitude,EntryHeading,ExitHeading,Hour,Weekend,Month,City,TimeFromFirstStop_p50,DistanceToFirstStop_p50, TotalTimeStopped_p50)

transformedTrain <- transformedTrain %>% mutate(
    JanAndMay = ifelse(Month == 1 | Month == 5, 1, 0),
    straightThrough = ifelse(EntryHeading == ExitHeading, 1, 0),
    rushHour = ifelse((Hour >= 6 & Hour <= 9) | (Hour >= 15 & Hour <= 18), 1, 0),
    hasWaitTime = ifelse(DistanceToFirstStop_p50 > 0 & TimeFromFirstStop_p50 > 0, 1, 0),
    LogTotalTimeStopped_p50 = ifelse(hasWaitTime == 1, log(TotalTimeStopped_p50), TotalTimeStopped_p50),
)

## Supporting functions

In [80]:
#' @description
#' Find group position for each elment of a vector
#'
#' @param values vector which should values that depends on the group
#' @param groupValues group values in the vector (hopefully unique)
#' @param tolerance for matching equality
#'
#' @return group membership for each element of the vector 'values'
#'
FindUniquePos=function(values,groupValues,tolerance=1.e-5)
{ ngroup = length(groupValues) # number of groups
temp = unique(groupValues)
if(length(temp)<ngroup)
{ cat("Won't work: non-unique group values\n"); return(0); }
npred = length(values)
group = rep(0,ngroup) # initialize as group 0
for(i in 1:ngroup)
{ #group[values==groupValues[i]]=i
igroup = (abs(values-groupValues[i])<tolerance)
group[igroup] = i # group label according to position in groupValues
}
if( any(group==0) ) cat("Warning: some values not matched to groupValues\n")
group
 }

In [23]:
#' interval score function for prediction intervals,
#' smaller value is better
#'
#' @description
#' interval score for prediction intervals
#'
#' @param predobj has 3 (or more) columns: pointprediction, predLB, predUB
#' @param actual vector of actual values (in holdout set, for example)
#' @param alpha level for prediction interval,
#' 1-alpha is expected coverage proportion if model is valid;
#' alpha=0.2 for 80% prediction intervals
#'
#' @return interval score
#'
intervalScore=function(predObj,actual,alpha)
{ n=nrow(predObj)
    ilow=(actual<predObj[,2]) # underestimation
    ihigh=(actual>predObj[,3]) # overestimation
    imid= (!ilow & !ihigh)
    sumlength=sum(predObj[,3]-predObj[,2]) # sum of lengths of prediction intervals
    sumlow=sum(predObj[ilow,2]-actual[ilow])*2/alpha
    sumhigh=sum(actual[ihigh]-predObj[ihigh,3])*2/alpha
    (sumlength+sumlow+sumhigh)/n # average length + average under/over penalties
}

## Regression trees

In [9]:
library(rpart); library(rpart.plot)

In [86]:
regTree = function(modelWrapper, dataset){
    smp_size <- floor(0.75 * nrow(dataset))
    train_ind <- sample(seq_len(nrow(dataset)), size = smp_size)

    train <- dataset[train_ind, ]
    holdo <- dataset[-train_ind, ]
    
    RegTree <- modelWrapper(train)
    
    meanByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,mean)
    # 50% predictions
    Q1ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,quantile,prob=0.25)
    Q2ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,median)
    Q3ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,quantile,prob=0.75)
    ByTNode=cbind(meanByTNode,Q1ByTNode,Q3ByTNode,Q2ByTNode)
    # 80% predictions
    Q180ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,quantile,prob=0.1)
    Q280ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,median)
    Q380ByTNode=tapply(train$TotalTimeStopped_p50,RegTree$where,quantile,prob=0.9)
    By80TNode=cbind(meanByTNode,Q180ByTNode,Q380ByTNode,Q280ByTNode)
    meanpredRegTree=predict(RegTree,newdata=holdo,type="vector")
    
    TNodeGroup=FindUniquePos(meanpredRegTree,meanByTNode)
    
    Q1predRegTree=Q1ByTNode[TNodeGroup]
    Q3predRegTree=Q3ByTNode[TNodeGroup]
    predIntRegTree=cbind(meanpredRegTree,Q1predRegTree,Q3predRegTree)

    Q180predRegTree=Q180ByTNode[TNodeGroup]
    Q380predRegTree=Q380ByTNode[TNodeGroup]
    predInt80RegTree=cbind(meanpredRegTree,Q180predRegTree,Q380predRegTree)
    
    cat("Avg Length | Interval Score | Coverage: Output for 50% prediction interval \n")
    avglengthTree=mean(predIntRegTree[,3]-predIntRegTree[,2])
    ISTree=intervalScore(predIntRegTree,holdo$TotalTimeStopped_p50,0.5)
    coverTree=mean(holdo$TotalTimeStopped_p50>=predIntRegTree[,2] & holdo$TotalTimeStopped_p50<=predIntRegTree[,3])
    cat("Reg:", avglengthTree,ISTree,coverTree,"\n")
    
    cat("Avg Length | Interval Score | Coverage: Output for 80% prediction interval \n")
    avglengthTree80=mean(predInt80RegTree[,3]-predInt80RegTree[,2])
    ISTree80=intervalScore(predInt80RegTree,holdo$TotalTimeStopped_p50,0.2)
    coverTree80=mean(holdo$TotalTimeStopped_p50>=predInt80RegTree[,2] & holdo$TotalTimeStopped_p50<=predIntRegTree[,3])
    cat("Reg:", avglengthTree80,ISTree80,coverTree80,"\n")
    }

In [81]:
basicRegressionTreeModelWrapper <- function(data){
    rpart(TotalTimeStopped_p50~LeftTurnAllowed+RightTurnAllowed+hasWaitTime+rushHour+straightThrough+Weekend+JanAndMay+City+RightTurn+LeftTurn, data=data, cp=0.0001, maxdepth= 6, minsplit=3)
}

In [87]:
r1 = regTree(basicRegressionTreeModelWrapper, transformedTrain)

Avg Length | Interval Score | Coverage: Output for 50% prediction interval 
Reg: 6.371841 12.85141 0.848592 
Avg Length | Interval Score | Coverage: Output for 80% prediction interval 
Reg: 12.38162 18.50328 0.8990317 


## Quantile Regression Forest

In [88]:
#install.packages("quantregForest")
library(quantregForest)

package 'quantregForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Alvi\AppData\Local\Temp\Rtmpw7EWt3\downloaded_packages


Warning message:
"package 'quantregForest' was built under R version 3.6.3"Loading required package: randomForest
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:gridExtra':

    combine

The following object is masked from 'package:dplyr':

    combine

The following object is masked from 'package:ggplot2':

    margin

Loading required package: RColorBrewer


In [89]:
intervalScorePlus=function(predObj,actual,alpha)
{ 
    n=nrow(predObj)
    ilow=(actual<predObj[,2]) # underestimation
    ihigh=(actual>predObj[,3]) # overestimation
    sumlength=sum(predObj[,3]-predObj[,2]) # sum of lengths of prediction intervals
    sumlow=sum(predObj[ilow,2]-actual[ilow])*2/alpha
    sumhigh=sum(actual[ihigh]-predObj[ihigh,3])*2/alpha
    avglength=sumlength/n
    IS=(sumlength+sumlow+sumhigh)/n # average length + average under/over penalties
    cover=mean(actual>= predObj[,2] & actual<=predObj[,3])
    summ=c(1-alpha,avglength,IS,cover)
    summ
}

In [92]:
quantRegForest = function(dataset){
    smp_size <- floor(0.75 * nrow(dataset))
    train_ind <- sample(seq_len(nrow(dataset)), size = smp_size)

    train <- dataset[train_ind, ]
    holdo <- dataset[-train_ind, ]
    
    logOrgTrain = train %>% select(IntersectionId, LeftTurnAllowed, RightTurnAllowed, City, straightThrough, rushHour, hasWaitTime,JanAndMay,LeftTurn, RightTurn, LogTotalTimeStopped_p50)
    orgTrain = train %>% select(IntersectionId, LeftTurnAllowed, RightTurnAllowed, City, straightThrough, rushHour, hasWaitTime, JanAndMay,LeftTurn, RightTurn, TotalTimeStopped_p50)

    p=10 # number of explanatory variables
    p1=p+1 # variable p+1 will be response variable
    
    ## remove observations with mising values
    logOrgTrain <- logOrgTrain[ !apply(is.na(orgTrain), 1,any), ]
    ## number of remining samples
    n <- 100000
    ## divide into training and test data
    indextrain <- sample(1:n,round(0.8*n),replace=FALSE)
    logXtrain <- logOrgTrain[ indextrain,2:p]
    logXtest <- logOrgTrain[-indextrain,2:p]
    logYtrain <- logOrgTrain[ indextrain,p1]
    logYtrain = logYtrain[[1]]
    logYtest <- logOrgTrain[-indextrain,p1]
    
    ## remove observations with mising values
    orgTrain <- orgTrain[ !apply(is.na(orgTrain), 1,any), ]
    Xtrain <- orgTrain[ indextrain,2:p]
    Xtest <- orgTrain[-indextrain,2:p]
    Ytrain <- orgTrain[ indextrain,p1]
    Ytrain = Ytrain[[1]]
    Ytest <- orgTrain[-indextrain,p1]
    
    Xtest = Xtest[1:20000,]
    Ytest = Ytest[1:20000,]
    logXtest = logXtest[1:20000,]
    logYtest = logYtest[1:20000,]
    
    qrf1 = quantregForest(x=Xtrain,y=Ytrain, mtry=8)
    qrf2 = quantregForest(x=logXtrain,y=logYtrain, mtry=8)
    
    pred1y = predict(qrf1, what=c(.1,.25,.5,.75,.9), newdata=Xtest)
    pred2ylog = predict(qrf2, what=c(.1,.25,.5,.75,.9), newdata=logXtest)
    
    pred2y = pred2ylog
    for(row in 1:nrow(pred2ylog)) {
        for(col in 1:ncol(pred2ylog)) {
            if (pred2ylog[row, col] != 0) {
                pred2y[row,col] = exp(pred2ylog[row, col])
            }
        }
    }
    
    actual1 = Ytest
    actual2=logYtest
    for(row in 1:nrow(logYtest)) {
        for(col in 1:ncol(logYtest)) {
            if (logYtest[row, col] != 0) {
                actual2[row,col] = exp(logYtest[row, col])
            }
        }
    }
    actual2 = actual2[[1]]
    actual1 = actual1[[1]]
    
    IS50qrf1=intervalScorePlus(pred1y[,c(3,2,4)],actual1,0.5)
    IS80qrf1=intervalScorePlus(pred1y[,c(3,1,5)],actual1,0.2)
    summaryTable=rbind(IS50qrf1,IS80qrf1)
    rownames(summaryTable)=c("trained on y","trained on y")
    colnames(summaryTable)=c("level","avglen","intervalScore","coverage")
    print(summaryTable)
    
    IS50qrf2=intervalScorePlus(pred2y[,c(3,2,4)],actual2,0.5)
    IS80qrf2=intervalScorePlus(pred2y[,c(3,1,5)],actual2,0.2)
    summaryTable2=rbind(IS50qrf2,IS80qrf2)
    rownames(summaryTable2)=c("trained on log(y)","trained on log(y)")
    colnames(summaryTable2)=c("level","avglen","intervalScore","coverage")
    print(summaryTable2)
}

In [93]:
qrf = quantRegForest(transformedTrain)

             level   avglen intervalScore coverage
trained on y   0.5  6.36705      12.94760  0.85095
trained on y   0.8 12.12758      18.75238  0.93870
                  level    avglen intervalScore coverage
trained on log(y)   0.5  6.359547      45.49245  0.85180
trained on log(y)   0.8 12.187747     100.07553  0.93925
